In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from gurobipy import GRB
import gurobipy as gp

## Cargo Problem

A cargo plane has three holds for storing cargo: front, centre and rear. These holds have the following limits on both weight and space:

**Holds**

| Hold   | Weight capacity (Tonnes) | Space capacity (Cubic Meters) |
|--------|--------------------------|-------------------------------|
| Front  | 10                       | 6800                          |
| Center | 16                       | 8700                          |
|  Back  | 8                        |  5300                         |


Furthermore, the weight of the cargo in the respective holds must be the same proportion of that hold’s weight capacity to maintain the balance of the plane. The following four cargoes are available for shipment on the next flight:

In [2]:
holds = [
    ['Front', 10, 6800],
    ['Center',16,8700],
    ['Back',8,5300],
]
holds = pd.DataFrame(holds)
holds.columns = ['Hold','Weight','Volume']

**Shipment**

| Cargo | Weight (Tonnes) | Volume (Cubic meters) | Profit / Tonne ($) |
|-------|-----------------|-----------------------|--------------------|
| C1    | 18              | 480                   | 310                |
| C2    | 15              | 650                   | 380                |
| C3    | 23              | 580                   | 350                |
| C4    | 12              | 390                   | 285                |

Any proportion of these cargoes can be accepted. The objective is to determine how much (if any) of each cargo C1, C2, C3 and C4 should be accepted and how to distribute each among the compartments so that the total profit for the flight is maximized.

Formulate the above problem as a linear program and solve it! Report the solution too.

In [3]:
shipment = [
    ['C1',18,480,310],
    ['C2',15,650,380],
    ['C3',23,580,350],
    ['C4',12,390,285],
]
shipment = pd.DataFrame(shipment)
shipment.columns = ['Cargo','Weight','Volume','Profit']

**Solution**

In [4]:
m = gp.Model("Cargo")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-11


In [5]:

'''
I don't know if I can allocate fraction number of Cargo like 1/2 of Unit C1,
If yes,
Please change to following code:

x = m.addMVar(shape = (4, 3), vtype=GRB.CONTINUOUS , name="decision")
'''

## Allocate decision variables
## x[i][j] -> put x[i][j] proportion of cargo i to hold j
x = m.addMVar(shape = (4, 3), vtype=GRB.INTEGER , name="decision")


In [6]:
## Add range constraints
for i in range(len(shipment)):
    for j in range(len(holds)):
        m.addConstr(
            0 <= x[i][j]
        )
for i in range(len(shipment)):
    m.addConstr(
        gp.quicksum(x[i]) <= shipment.loc[i, 'Weight']
    )

In [7]:
## Add resource constraints
for j, row in holds.iterrows():
    m.addConstr(
        gp.quicksum(
            x[k][j]
            for k in range(len(shipment))
        ) <= row['Weight']
    )
    
    m.addConstr(
        gp.quicksum(
            x[k][j] * shipment.loc[k, 'Volume']
            for k in range(len(shipment))
        ) <= row['Volume']
    )
    

In [8]:
## Add objective
m.setObjective(
    gp.quicksum(
        gp.quicksum(x[k]) * shipment.loc[k, 'Profit']
        for k in range(len(shipment))
    )
    ,GRB.MAXIMIZE
)


In [9]:
## Optimization
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 22 rows, 12 columns and 48 nonzeros
Model fingerprint: 0x34228220
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [3e+02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 9e+03]
Found heuristic solution: objective 11280.000000
Presolve removed 14 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 12 columns, 28 nonzeros
Variable types: 0 continuous, 12 integer (0 binary)
Found heuristic solution: objective 11360.000000

Root relaxation: objective 1.215158e+04, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 12151.5789    0    2 11360.0000 12151.5789  6.97%     -

In [10]:
for i in range(len(shipment)):
    for j in range(len(holds)):
        print('Distribute %f %s to %s'%(x[i][j].x, shipment.loc[i, 'Cargo'], holds.loc[j, 'Hold']))

Distribute 0.000000 C1 to Front
Distribute 2.000000 C1 to Center
Distribute 0.000000 C1 to Back
Distribute 7.000000 C2 to Front
Distribute 0.000000 C2 to Center
Distribute 8.000000 C2 to Back
Distribute 3.000000 C3 to Front
Distribute 12.000000 C3 to Center
Distribute 0.000000 C3 to Back
Distribute 0.000000 C4 to Front
Distribute 2.000000 C4 to Center
Distribute 0.000000 C4 to Back


In [ ]:
total = sum([sum([x[i][j].x for j in range(len(holds))])*shipment.loc[i, 'Profit'] for i in range(len(shipment))])

: 

## Production Planning

| Month  | 3    | 4    | 5    | 6    | 7    | 8    |
|--------|------|------|------|------|------|------|
| Demand | 5000 | 6000 | 6500 | 7000 | 8000 | 9500 |

The company currently has in stock: 1000 units which were produced in month 2; 2000 units which were produced in month 1; 500 units which were produced in month 0.

The company can only produce up to 6000 units per month and the managing director has stated that stocks must be built up to help meet demand in months 5, 6, 7 and 8. Each unit produced costs 15 doller and the cost of holding stock is estimated to be 0.75 doller per unit per month (based upon the stock held at the beginning of each month).

The company has a major problem with deterioration of stock in that the stock inspection which takes place at the end of each month regularly identifies ruined stock (costing the company $25 per unit). It is estimated that, on average, the stock inspection at the end of month t will show that 11\% of the units in stock which were produced in month t are ruined; 47\% of the units in stock which were produced in month t-1 are ruined; 100\% of the units in stock which were produced in month t-2 are ruined. The stock inspection for month 2 is just about to take place.

The company wants a production plan for the next six months that avoids stock outs (and minimizes costs).

Formulate their problem as a linear program and solve it! Report the solution too.
Add submission


In [11]:
demand = [0,0,0,5000,6000,6500,7000,8000,9500]

In [12]:
m = gp.Model("Planning")

In [13]:
## Allocate decision variables
x = m.addMVar(shape=(len(demand)), vtype=GRB.INTEGER , name="decision")
# x = m.addMVar(shape=(len(demand)), vtype=GRB.CONTINUOUS , name="decision")

In [14]:
## Add constraints on max/min production values
m.addConstr(x[0] == 500)
m.addConstr(x[1] == 2000)
m.addConstr(x[2] == 1000)

for i in range(3, x.shape[0]):
    m.addConstr(
        0 <= x[i]
    )
    m.addConstr(
        x[i] <= 8000
    )

In [ ]:
## Add intermediate variable y to represent stock allocations
## y[i][0] -> production for current month, y[i][1] -> residual from month i-1, y[i][2] -> residual from month i-2
y = m.addMVar(shape=(len(demand), 3), vtype=GRB.INTEGER , name="alloc")

: 

In [15]:
# ## Add intermediate variable y to represent residual stock
# ## y[i][0] -> residual from month i-1, y[i][1] -> residual from month i-2
# y = m.addMVar(shape=(len(demand), 2), vtype=GRB.INTEGER , name="decision")
# # y = m.addMVar(shape=(len(demand), 2), vtype=GRB.CONTINUOUS , name="decision")

# # 500
# m.addConstr(
#     y[0][0] + y[0][1] == x[0] - demand[0]
# )
# # 2445
# m.addConstr(
#     y[1][0] + y[1][1] == x[1] + 0.89 * y[0][0] - demand[1]
# )

# for i in range(2, x.shape[0]):
#     m.addConstr(
#         y[i][0] + y[i][1] == x[i] + 0.89 * y[i - 1][0] + 0.53 * y[i - 2][1] - demand[i]
#     )

# ## Add demand constraints
# for i in range(x.shape[0]):
#     m.addConstr(
#         y[i][0] >= 0
#     )
#     m.addConstr(
#         y[i][1] >= 0
#     )

# ## Add objective
# '''
# I don't know if we also do inspection in month 9?
# If yes,
# Please change to following code:

# m.setObjective(
#     gp.quicksum(y) * 0.75 + 
#     (gp.quicksum(x) - sum(demand) + y[y.shape[0] - 1]) * 25
#     ,GRB.MINIMIZE
# )
# '''

# # m.setObjective(
# #     gp.quicksum(y) * 0.75 + 
# #     (gp.quicksum(x) - sum(demand)) * 25
# #     ,GRB.MINIMIZE
# # )

# ## Optimization
# try: 
#     m.optimize()
# except gp.GurobiError as E:
#     print("Optimize failed", E)

In [16]:
'''
😭 Just noticed the slack channel that professor mentioned:

(1) ruined only
(2) the ruined stock applies to the quantities that 
    roll over to the next month after serving the demand for this month.
'''

'\n😭 Just noticed the slack channel that professor mentioned:\n\n(1) ruined only\n(2) the ruined stock applies to the quantities that \n    roll over to the next month after serving the demand for this month.\n'

In [17]:
'''
😭 I should attend the class man...

Per class, production limit for problem 2 is 8000
'''

'\n😭 I should attend the class man...\n\nPer class, production limit for problem 2 is 8000\n'

In [18]:
## Add constraint

for i in range(x.shape[0]):
    m.addConstr(
        gp.quicksum(x[:i+1]) - sum(demand[:i+1]) >= 0
    )

## Add objective
m.setObjective(
    gp.quicksum(
        (gp.quicksum(x[:i+1]) - sum(demand[:i+1]))     * 0.75 +
        (
            (gp.quicksum(x[:i]) - sum(demand[:i]))     * 0.11 + 
            (gp.quicksum(x[:i-1]) - sum(demand[:i-1])) * 0.47 +
            (gp.quicksum(x[:i-2]) - sum(demand[:i-2])) * 1
        ) * 25
        for i in range(x.shape[0])
    )
    ,GRB.MINIMIZE
)

In [19]:
## Optimization
try: 
    m.optimize()
except gp.GurobiError as E:
    print("Optimize failed", E)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 24 rows, 9 columns and 60 nonzeros
Model fingerprint: 0xcd85d52a
Variable types: 0 continuous, 9 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+02, 4e+04]
Presolve removed 22 rows and 5 columns
Presolve time: 0.00s
Presolved: 2 rows, 4 columns, 6 nonzeros
Variable types: 0 continuous, 4 integer (0 binary)
Found heuristic solution: objective 1207258.5000

Root relaxation: objective 4.025000e+05, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    402500.00000 402500.000  0.00%     -    0s

Explored 1 nodes (1 simplex iterations) in 0.01

In [20]:
for i in range(x.shape[0]):
    print('Produce %f in %d-th month'%(x[i].x, i))

Produce 500.000000 in 0-th month
Produce 2000.000000 in 1-th month
Produce 1000.000000 in 2-th month
Produce 1500.000000 in 3-th month
Produce 6000.000000 in 4-th month
Produce 7000.000000 in 5-th month
Produce 8000.000000 in 6-th month
Produce 8000.000000 in 7-th month
Produce 8000.000000 in 8-th month


In [21]:
sum([x[i].x for i in range(x.shape[0])]) == sum(demand)

True